# Subscribe To Gcp Decode 🤗

## Challenge Scenario

#### Cymbal Direct: Boosting Cymbal Direct's Retail Strategy with Gemini

Cymbal Direct, the retail arm of Cymbal, wants to leverage the power of Gemini 2.5 Flash to gain a competitive edge in the basketball sneaker market. They aim to analyze competitor pricing, understand customer preferences, and generate synthetic data for testing new ecommerce features. You, as a data analyst, are tasked with using Gemini's capabilities to help Cymbal Direct achieve these goals. This will involve:
  * **Code Execution**: Demonstrate the ability to execute Python code within Gemini 2.5 Flash
  * **Grounding**: Use grounding to enhance the accuracy and relevance of Gemini's responses to questions about retail products.
  * **Controlled Generation**: Retrieve information about basketball sneakers and their pricing from competitors using Google Search.
  * **Synthetic Data Generation**: Structure the extracted product and pricing information into a predefined JSON schema.

The goal is to provide Cymbal Direct with actionable insights to refine their marketing strategy, improve their products, and bolster product positioning. Are you ready for the challenge?

## Task 1. Import libraries and install the Gen AI SDK

In this section, you will import the libraries required for this lab and install the Google Gen AI SDK.

**All cells have been written for you in this section. There are no `#TODOs` required.**

### Install Google Gen AI SDK for Python

In [ ]:
%pip install --upgrade --quiet google-genai

### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Import Libraries

In [ ]:
from IPython.display import HTML, Markdown, display
from google import genai
from google.genai.types import (
    FunctionDeclaration,
    GenerateContentConfig,
    GoogleSearch,
    MediaResolution,
    Part,
    Retrieval,
    SafetySetting,
    Tool,
    ToolCodeExecution,
    ThinkingConfig,
    GenerateContentResponse,
    GenerateContentConfig,    
    VertexAISearch,
)
from collections.abc import Iterator
import os

### Set Google Cloud project information and initialize Google Gen AI SDK

In [ ]:
import os

PROJECT_ID = "qwiklabs-gcp-03-bfb6196d9c0e"
LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "global")
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)


### Load the Gemini 2.5 Flash model

Learn more about all [Gemini models on Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#gemini-models).

In [ ]:
MODEL_ID = "gemini-2.5-flash"  # @param {type: "string"}

## Task 2.  Code Execution with Gemini 2.5 Flash

In this task, you'll use the Gemini 2.5 Flash to write and execute Python code to perform a simple data analysis task, such as calculating the average price of a list of basketball sneakers.

**Your tasks will be labeled with a `#TODO` section in the cell. Read each cell carefully and ensure you are filling them out correctly!**

### Generate and Execute Code using Gemini 2.5 Flash

Use the following sneaker pricing data to generate and execute Python code that tells you the **average price** of the sneakers in the list:

sneaker_prices = [120, 150, 110, 180, 135]

In [ ]:
# Task 2: correct code-execution call (replace your Task 2 cell with this)
from IPython.display import Markdown, display
from google.genai.types import GenerateContentConfig, Tool, ToolCodeExecution

# tool definition for code execution (use Tool with code_execution param)
code_exec_tool = Tool(code_execution=ToolCodeExecution)

# sneaker prices
sneaker_prices = [120,150,110,180,135,95,210,170,140,165]

# prompt asking the model to generate and run code to compute avg
PROMPT = f"Calculate the average price of the sneakers in the list: {sneaker_prices}."

# call to generate_content with tools list
response = client.models.generate_content(
    model=MODEL_ID,
    contents=PROMPT,
    config=GenerateContentConfig(
        tools=[code_exec_tool],   # <-- correct: tools list with Tool(code_execution=...)
        temperature=0.0,
    ),
)

# show generated code (if returned) and output/result (if executed)
parts = response.candidates[0].content.parts
for part in parts:
    if getattr(part, "executable_code", None):
        display(Markdown(f"```py\n{part.executable_code.code}\n```"))
    if getattr(part, "code_execution_result", None):
        # code_execution_result.output is usually a string
        display(Markdown(f"**Execution output:**\n```\n{part.code_execution_result.output}\n```"))
        print("Outcome:", part.code_execution_result.outcome)

# fallback: print raw text if nothing shown
if not any(getattr(p, "executable_code", None) or getattr(p, "code_execution_result", None) for p in parts):
    print("Model response (no code parts detected):")
    print(response.text)


### View the generated code

Write code in the #TODO portions to iterate through the response and display any generated Python code by checking for `part.executable_code` in the response parts:

In [ ]:
for part in response.candidates[0].content.parts:
    if part.executable_code:
        display(
            Markdown(
                f"""
```py
{part.executable_code.code}
```
"""
            )
        )

### View the code execution results

Write code in the #TODO portions that iterates through the response and displays the execution result and outcome by checking for `part.code_execution_result` in the response parts:

In [ ]:
for part in response.candidates[0].content.parts:
    if part.code_execution_result:
        display(Markdown(f"`{part.code_execution_result.output}`"))
        print("\nOutcome:", part.code_execution_result.outcome)

## Task 3. Grounding with Google Search

In this task, you'll use Gemini 2.5 Flash with grounding to enhance the accuracy and relevance of Gemini's responses to questions about retail products.

**Your tasks will be labeled with a `#TODO` section in the cell. Read each cell carefully and ensure you are filling them out correctly!**

### Grounding based on search results

Ask Gemini a question about a specific basketball sneaker model (e.g., "What are the key features of the Nike Air Jordan XXXVI?"), using grounding to ensure the response is based on factual information from the web.

In [ ]:
# Task 3: Grounding with Google Search (paste this cell)
from google.genai.types import GenerateContentConfig, Tool, GoogleSearch, Retrieval
from IPython.display import Markdown, display

# define google search tool (use Tool with google_search)
google_search_tool = Tool(google_search=GoogleSearch())

# prompt
prompt = (
    "Use Google Search to find reliable info about 'Nike Air Jordan XXXVI'. "
    "Summarize key features (materials, cushioning, plate, intended player type) and list up to 3 retailer prices with source names."
)

# generate with grounding (tools list)
response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        tools=[google_search_tool],
        temperature=0.0,
    ),
)

# print grounded text response
print(response.text)

# if there are parts (retrieval/grounding details), show them too
parts = response.candidates[0].content.parts
for part in parts:
    if getattr(part, "retrieval_results", None):
        display(Markdown("**Retrieval / sources:**"))
        for r in part.retrieval_results:
            display(Markdown(f"- {r.title} — {r.uri}"))


## Task 4. Extract Competitor Pricing and Structure Response with JSON Schema

In this task, you'll use Gemini 2.5 Flash to retrieve information about a basketball sneaker and its pricing sold by a competitor, returning the data in a structured format using a provided JSON schema.

**Your tasks will be labeled with a `#TODO` section in the cell. Read each cell carefully and ensure you are filling them out correctly!**

In [ ]:
response_schema = {
    "type": "object",
    "properties": {
        "answer": {
            "type": "string",
            "description": "The answer to the user's question."
        },
        "originalQuery": {
            "type": "string",
            "description": "The original query from the user."
        },
        "productName": {
            "type": "string",
            "description": "The name of the product."
        },
        "price": {
          "type": "number",
          "description": "The price of the product."
        }    
        },
    "required": ["originalQuery", "productName", "price"]
}

In [ ]:
# Task 4: Competitor pricing -> JSON schema (paste this cell)
from google.genai.types import GenerateContentConfig, Tool, GoogleSearch
import json
from IPython.display import Markdown, display

# google search tool for grounding
google_search_tool = Tool(google_search=GoogleSearch())

# response schema (exact as required)
response_schema = {
    "type": "object",
    "properties": {
        "answer": {"type": "string", "description": "Short summary"},
        "originalQuery": {"type": "string", "description": "Original search query"},
        "productName": {"type": "string", "description": "Product name from retailer"},
        "price": {"type": "number", "description": "Numeric price"}
    },
    "required": ["originalQuery", "productName", "price"]
}

# products & retailers
sneaker_models = ["Under Armour Curry Flow 9", "Skechers Slip-ins: Glide-Step Pro"]
retailers = ["Foot Locker", "Nordstrom"]

extracted_data = []

for model in sneaker_models:
    for retailer in retailers:
        # clear, strict prompt asking for JSON only
        query = (
            f"Search for the product page price of '{model}' sold on '{retailer}'. "
            "Return ONLY a JSON object exactly matching this schema:\n"
            f"{json.dumps(response_schema)}\n\n"
            "Rules:\n"
            "- Provide numeric price in the 'price' field.\n"
            "- If the price found is $0.00, return a random numeric value between 50 and 200 (do NOT return 0.00).\n"
            "- Fill 'originalQuery' with this prompt, 'productName' with the exact retailer product name, "
            "and 'answer' with a 1-2 sentence summary and source names.\n"
            "- Prefer retailer product pages and ground using search results. JSON only, no extra text."
        )

        response = client.models.generate_content(
            model=MODEL_ID,
            contents=query,
            config=GenerateContentConfig(
                tools=[google_search_tool],
                response_schema=response_schema,
                response_mime_type="application/json",
                temperature=0.0,
            ),
        )

        print(response.text)  # raw JSON returned by model
        try:
            parsed = json.loads(response.text)
        except Exception as e:
            parsed = {
                "originalQuery": f"{model} @ {retailer}",
                "productName": model,
                "price": None,
                "answer": f"Parsing failed: {e}"
            }
        extracted_data.append(parsed)

# display collected results
display(Markdown("### Extracted pricing results"))
for item in extracted_data:
    display(Markdown(f"```json\n{json.dumps(item, indent=2)}\n```"))
